# Online Inference

This notebook simulates an online inference pipeline by publishing data to a Kinesis stream which is picked up by an associated Lambda function and this Lambda function joins this data with additional data from an online feature store and then invokes a SageMaker endpoint to get inference in real-time.


In [ ]:
from pathlib import Path
import pandas as pd
import logging
import boto3
import json
import sys
import os

In [ ]:
# import from a different path
path = Path(os.path.abspath(os.getcwd()))
package_dir = f'{str(path.parent)}/utils'
print(package_dir)
sys.path.insert(0, package_dir)
import utils

## Setup logging

In [ ]:
logger = logging.getLogger('__name__')
logging.basicConfig(format="%(asctime)s,%(filename)s,%(funcName)s,%(lineno)s,%(levelname)s,p%(process)s,%(message)s", level=logging.INFO)       

## Global constants

In [ ]:
# global constants
STACK_NAME = "expedia-feature-store-demo-v2"
LOCAL_DATA_DIR = "../data"
DESTINATION_FEATURES = "pc1,pc2,pc3"
PREDICTED_VARIABLE = "hotel_cluster_predicted"
RECORDS_TO_STREAM = 5
PK = 'user_id' # partition key for kinesis stream

## Setup config variables

In [ ]:
# read output variables from cloud formation stack, these will be used as parameters throughout
# the code
data_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'DataBucketName')
model_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'MLModelBucketName')
athena_query_results_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'AthenaQueryResultsBucketName')
feature_store_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'FeatureStoreBucketName')
hotel_cluster_prediction_fn_arn = utils.get_cfn_stack_outputs(STACK_NAME, 'HotelClusterPredictionFunction')
hotel_cluster_prediction_ddb_table_name = utils.get_cfn_stack_outputs(STACK_NAME, 'HotelClusterPredictionsTableName')

logger.info(f"data_bucket_name={data_bucket_name},\nathena_query_results_bucket_name={athena_query_results_bucket_name},\n"
            f"model_bucket_name={model_bucket_name}\nfeature_store_bucket_name={feature_store_bucket_name},\n"
            f"hotel_cluster_prediction_fn_arn={hotel_cluster_prediction_fn_arn}\nhotel_cluster_prediction_ddb_table_name={hotel_cluster_prediction_ddb_table_name}")

In [ ]:
# read outputs from previous notebooks that are needed by this notebook.
# these are available as local files.
customer_inputs_fg_name = utils.read_param("customer_inputs_fg_name")
destinations_fg_name = utils.read_param("destinations_fg_name")
customer_inputs_fg_table = utils.read_param("customer_inputs_fg_table")
destinations_fg_table = utils.read_param("destinations_fg_table")
customer_inputs_fg_name = utils.read_param("customer_inputs_fg_name")
ml_model_endpoint_name = utils.read_param("endpoint_name")

# read params from the cloud formation stack
raw_data_dir = utils.get_cfn_stack_parameters(STACK_NAME, 'RawDataDir')
app_name = utils.get_cfn_stack_parameters(STACK_NAME, 'AppName')

training_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingDatasetFileName')
test_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'TestDatasetFileName')
validation_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'ValidationDatasetFileName')

training_job_instance_type = utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingJobInstanceType')
if training_job_instance_type is None:
    training_job_instance_type = "ml.m5.xlarge"
training_job_instance_count = int(utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingJobNodeInstanceCount'))

model_ep_instance_type = utils.get_cfn_stack_parameters(STACK_NAME, 'ModelEndpointInstanceType')
model_ep_instance_count = int(utils.get_cfn_stack_parameters(STACK_NAME, 'ModelEndpointInstanceCount'))

customer_input_stream_name = utils.get_cfn_stack_parameters(STACK_NAME, 'CustomerInputStreamName')
            
logger.info(f"customer_inputs_fg_table={customer_inputs_fg_table},\ndestinations_fg_table={destinations_fg_table},\n"
            f"customer_inputs_fg_name={customer_inputs_fg_name},\ndestinations_fg_name={destinations_fg_name}\n"
            f"raw_data_dir={raw_data_dir},\ntraining_dataset_fname={training_dataset_fname},\n"
            f"test_dataset_fname={test_dataset_fname},\nvalidation_dataset_fname=-{validation_dataset_fname}\n"
            f"training_job_instance_type={training_job_instance_type},\ntraining_job_instance_count={training_job_instance_count},\n"
            f"model_ep_instance_type={model_ep_instance_type},\nmodel_ep_instance_count={model_ep_instance_count},\ncustomer_input_stream_name={customer_input_stream_name}")

## Update the lambda function

The lambda function handler for the Kinesis stream needs to be updated with the SageMaker endpoint name (this name was not available at the time of deploying the Lambda via the cloud formation template).

In [ ]:
# clients for the services we are going to use
lambda_client = boto3.client('lambda')

In [ ]:
# we use the environment variables for the lambda as a mechanism for passing config values. 
# Sagemaker endpoint name, the destinations feature group name etc are all available 
# as variables in this notebook (read already from local files). In a production environment
# these would be read from Parameter Store.

logger.info(f'updating Lambda function with ARN={hotel_cluster_prediction_fn_arn} to use ML model endpoint: {ml_model_endpoint_name}')
variables = lambda_client.get_function_configuration(FunctionName=hotel_cluster_prediction_fn_arn)['Environment']['Variables']
variables['ENDPOINT_NAME'] = ml_model_endpoint_name
variables['FG_NAME'] = destinations_fg_name
variables['DDB_TABLE_NAME'] = hotel_cluster_prediction_ddb_table_name
variables['ONLINE_FEATURE_GROUP_KEY'] = 'srch_destination_id'
variables['ONLINE_FEATURE_GROUP_FEATURES_OF_INTEREST'] = DESTINATION_FEATURES
variables['PREDICTED_VARIABLE'] = PREDICTED_VARIABLE

resp = lambda_client.update_function_configuration(
    FunctionName=hotel_cluster_prediction_fn_arn,
      Environment={
        'Variables': variables
    }
)

## Stream test data
At this point we are all set to stream test data on the Kinesis data stream. This test data is available in a local file.

In [ ]:
# read test data from local file
fpath = os.path.join(LOCAL_DATA_DIR, test_dataset_fname)
df = pd.read_csv(fpath)
logger.info(f"read test data from {fpath}, dataframe shape is {df.shape}")
df.head()

In [ ]:
# print out the first record just for debug purposes
record = json.loads(df.to_json(orient='records'))[0]
record

In [ ]:
# stream each row of the dataframe as a json to the Kinesis data stream
kinesis_client = boto3.client('kinesis')

for record in json.loads(df.to_json(orient='records'))[:RECORDS_TO_STREAM]:
    data = json.dumps(record)
    logger.info(f"Sending data, record.{PK}={record[PK]}...")
    response = kinesis_client.put_record(StreamName = customer_input_stream_name,
                                         Data = data,
                                         PartitionKey = PK)

    if (response['ResponseMetadata']['HTTPStatusCode'] != 200):
        logger.error("ERROR: Kinesis put_record failed: \n{}".format(json.dumps(response)))
    else:
        logger.info("data sent successfully...")

## Check the DynamoDB table

Now check the DynamoDB table (ExpediaPerCustomerHotelClusterPredictionsunless changed when deploying the Cloud Formation template) for new data inserted corresponding to the records streamed in the previous step. Look for the hotel_cluster_predicted field, this field contains the prediction from the SageMaker model endpoint.

Also check the logs of the Lambda function to see that it got invokes for every record put on the Kinesis stream.

<img src="../images/ddb_table.png">Data Profile</img>